In [2]:
import json

In [3]:
with open('vicuna_experiments/vicuna_context/answers_with_context.json', 'r') as f:
    with_context = json.loads(f.read())
    
with open('vicuna_experiments/vicuna_without_context/answers_without_context.json', 'r') as f:
    without_context = json.loads(f.read())

In [7]:
with_context[0]

"Context:\n1. The Assistant has vision problems and has called 911 in the past due to severe headaches.\n2. The Assistant used to have six cats and is now married to a friend from high school.\n3. The Human is from Virginia and is not currently married.\n\n Facts about Human:\nHuman is not married and lives in Virginia.\n\nFacts about Assistant:\nWorks from home\nHas trouble seeing\nHas 200 feet vision\nGets bad headaches\nHas called 911 before\nUsed to have 6 cats\nLives in Alaska\nAssistant has vision problems and gets headaches.\nAssistant is from Alaska and got married to a friend from high school.\n\n Dialog:\nHuman: How are you and your cats doing today over in Alaska?\nAssistant: Good thanks. How are you and your dad in Virginia? \nHuman: We are doing pretty good thank you for asking. I hope you don't have any headaches today!\nAssistant: I am having a headache right now. My vision is messed up and I can hardly see. "

In [49]:
split_and_fix = lambda x: [row for row in x.split('\n') if row]

collected_answers = [{'context': split_and_fix(context_dialog)[-1], 'simple': split_and_fix(simple_dialog)[-1]} 
                     for context_dialog, simple_dialog in zip(with_context, without_context)]

In [50]:
cleaned_dialogs = [dialog.replace(answers['context'], '') for dialog, answers in zip(with_context, collected_answers)]

In [56]:
def shuffle_answers(context_answer, simple_answer):
    from random import shuffle
    
    answers = [context_answer, simple_answer]
    shuffle(answers)

    if answers[0] == context_answer:
        contexted_answer = 'Answer A'
    else:
        contexted_answer = 'Answer B'
    
    return {'Answer A': answers[0], 'Answer B': answers[1], 'Contexted Answer': contexted_answer}

annotation_dataset = [{'dialog': dialog, **shuffle_answers(answers['context'], answers['simple'])} 
                      for dialog, answers in zip(cleaned_dialogs, collected_answers)]

In [19]:
200*90

18000

In [62]:
with open('vicuna_experiments/annotation_dataset.json', 'w') as f:
    json.dump(annotation_dataset, f)

In [6]:
with open('vicuna_experiments/annotation_dataset.json', 'r') as f:
    annotation_data = json.loads(f.read())

In [2]:
with open('vicuna_experiments/annotated_data.jsonl', 'r') as f:
    annotated_data = f.readlines()
    annotated_data = [json.loads(data) for data in annotated_data]

In [7]:
sum([data['Answer'] == data['Contexted Answer'] or data['Answer'] == 'Both answers are relevant' 
     for data in annotated_data]) / len(annotated_data)

0.5838926174496645

In [8]:
sum([data['Answer'] == data['Contexted Answer'] for data in annotated_data]) /  \
        len([data for data in annotated_data if data['Answer'] in ['Answer A', 'Answer B']])

0.5897435897435898

In [9]:
idx = 46
print(annotation_data[idx]['dialog'])
print('Answer A:\n', annotation_data[idx]['Answer A'])
print('Answer B:\n', annotation_data[idx]['Answer B'])
print('Contexted Answer:\n', annotation_data[idx]['Contexted Answer'])

Context:
1. Assistant enjoys eating meat and mentions riding in a black sexy car with their pet panda.
2. Human mentions liking cheddar on their biscuits and riding a bike.
3. Human also mentions being an insurance salesman and having heightened senses and knowledge.
* Human is interested in seeing Assistant's Kung Fu Panda, Chewy.
* Human is available to meet up later that night to grill out and meet Chewy.
* Human is looking forward to meeting Chewy.

 Facts about Human:
Human likes cheddar cheese on biscuits
Human enjoys riding bikes
Human is an insurance salesman
Human has heightened senses and knowledge
Human is interested in visiting Chewy the panda
Human's bike is currently being repaired and they are unable to ride over to see Chewy
Human is free to visit Chewy tonight and will bring a salad to contribute to dinner

Facts about Assistant:
Assistant has insurance for their car but not for their pet panda because the panda knows martial arts
Assistant enjoys listening to rap musi